In [0]:
%run "./startup" 

In [0]:
params = WidgetParams({
    "sr_folio": str,    
    "sr_tipo_archivo": str,
    "sr_id_archivo": str,
    "sr_subproceso": str
})
# Validar widgets
params.validate()

In [0]:
conf = ConfManager()

#Archivos SQL
query = QueryManager()

#Conexion a base de datos
db = DBXConnectionManager()

file_manager = FileManager(err_repo_path=conf.err_repo_path)

In [0]:
#Query Extrae informacion 

statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0010_TRANF_FOV.sql",
    sr_folio = params.sr_folio,
    sr_tipo_archivo = params.sr_tipo_archivo,
    sr_id_archivo = params.sr_id_archivo,
    sr_subproceso = params.sr_subproceso
)

db.write_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}", db.read_data("default", statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}"))

In [0]:
statement = query.get_statement(
    "TTAFOTRAS_ETL_TRANSF_FOVISSSTE_DELETE.sql",
    sr_folio=params.sr_folio,
    sr_id_archivo = params.sr_id_archivo,
    sr_tipo_archivo = params.sr_tipo_archivo
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
 
 #INSERTA EN LA TABLA

table_name = "CIERREN_ETL.TTAFOTRAS_ETL_TRANSF_FOVISSSTE"

db.write_data(db.read_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}"), table_name, "default", "append")

In [0]:
#Se elimina la Delta para liberar espacio
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}")

In [0]:
statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0010_TRANF_FOV_TEST_SELECT.sql",
    sr_folio = params.sr_folio
)

db.write_delta(f"DELTA_TRA_TEST_DEBUG_{params.sr_folio}", db.read_data("default", statement), "overwrite")
display(db.read_delta(f"DELTA_TRA_TEST_DEBUG_{params.sr_folio}"))

In [0]:
#Se ejecuta el UPDATE


statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0020_COMPARA_TRANF_FOV_02_UPDATE.sql",
    sr_folio=params.sr_folio,
    sr_tipo_archivo=params.sr_tipo_archivo
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#Query Extrae informacion 

statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0020_COMPARA_TRANF_FOV_03.sql",
    sr_folio = params.sr_folio,
    sr_tipo_archivo = params.sr_tipo_archivo,
    sr_id_archivo = params.sr_id_archivo,
    sr_subproceso = params.sr_subproceso
)

db.write_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}", db.read_data("default", statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}"))

In [0]:
#Se eliminan los registros de la tabla auxiliar
statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0020_COMPARA_TRANF_FOV_03_DELETE.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#INSERTA EN LA TABLA

table_name = "CIERREN_DATAUX.TTAFOTRAS_ETL_TRANSF_FOVISSSTE_AUX"

db.write_data(db.read_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}"), table_name, "default", "append")


In [0]:
#Se ejecuta el merge


statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0020_COMPARA_TRANF_FOV_03_MERGE.sql",
    sr_folio=params.sr_folio
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#Se elimina la Delta para liberar espacio
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}")

#Se eliminan los registros de la tabla auxiliar
statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0020_COMPARA_TRANF_FOV_03_DELETE.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#Query Extrae informacion 

statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0020_COMPARA_TRANF_FOV_02.sql",
    sr_folio = params.sr_folio,
    sr_tipo_archivo = params.sr_tipo_archivo,
    sr_id_archivo = params.sr_id_archivo,
    sr_subproceso = params.sr_subproceso
)

db.write_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}", db.read_data("default", statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}"))

In [0]:
# Cargar tabla Delta en un DataFrame
df = spark.read.format("delta").table(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}")

# Mostrar los primeros 100 registros
df.show(100)

In [0]:
import os

#If p_SR_SUBPROCESO = 364 And p_SR_TIPO_ARCHIVO = "06" Then "Comparacion_Archivo_Saldo" Else "Comparacion_Archivo_Rechazo"
sr_nombrearch = "Comparacion_Archivo_Rechazo"
#output_path = f"/NCI/RCTRAS/VIV/INF/{params.sr_folio}_{params.sr_nombrearch}.csv"
output_path = f"{params.sr_folio}_{sr_nombrearch}"

full_path = SETTINGS.GENERAL.EXTERNAL_LOCATION + SETTINGS.GENERAL.PATH_RCTRAS + '/' + SETTINGS.GENERAL.PATH_INF + '/' + os.path.basename(output_path) +'.csv'
# Genera el archivo y decide si calcular MD5 o no
file_manager.generar_archivo_ctindi(
   df_final=df,  # DataFrame que se va a guardar
   full_file_name=full_path,
   header=True,
   calcular_md5=False  # Cambia a False si no quieres calcular el MD5
)


In [0]:
#borrar la delta 
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}")

In [0]:
#Query Extrae informacion 

statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0020_COMPARA_TRANF_FOV_01.sql",
    sr_folio = params.sr_folio,
    sr_tipo_archivo = params.sr_tipo_archivo,
    sr_id_archivo = params.sr_id_archivo,
    sr_subproceso = params.sr_subproceso
)

db.write_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}", db.read_data("default", statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}"))

In [0]:

#Se eliminan los registros de la tabla auxiliar
statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0020_COMPARA_TRANF_FOV_01_DELETE.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#INSERTA EN LA TABLA

table_name = "CIERREN_DATAUX.TTAFOTRAS_TRANS_FOVISSSTE_AUX"

db.write_data(db.read_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}"), table_name, "default", "append")

In [0]:
#Se ejecuta el merge


statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0020_COMPARA_TRANF_FOV_01_MERGE.sql",
    sr_folio=params.sr_folio
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#Se elimina la Delta para liberar espacio
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}")

#Se eliminan los registros de la tabla auxiliar
statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0020_COMPARA_TRANF_FOV_01_DELETE.sql",
    sr_folio=params.sr_folio,
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)